In [1]:
from PySide6.QtCore import (QCoreApplication, QDate, QDateTime, QLocale,
    QMetaObject, QObject, QPoint, QRect, QTimer, QTimer,
    QSize, QTime, QUrl, Qt)
from PySide6.QtGui import (QBrush, QColor, QConicalGradient, QCursor,
    QFont, QFontDatabase, QGradient, QIcon, QIntValidator,
    QImage, QKeySequence, QLinearGradient, QPainter,
    QPalette, QPixmap, QRadialGradient, QTransform)
from PySide6.QtWidgets import (QApplication, QComboBox, QFrame, QHBoxLayout,
    QLabel, QLineEdit, QMainWindow, QPushButton, QFileDialog, QDialog, QMessageBox,
    QSizePolicy, QWidget)
import sys
import time
import datetime
import json
from pathlib import Path

import cv2
from PIL import ImageFont, ImageDraw, Image
from ui_yolo11 import *
from yolo import *
from alpr_kor import *
from rknnlite.api import RKNNLite

TASKS = ["detect", "pose", "seg", "obb"]
DATASETS = ["COCO", "FIRE", "CRACK", "LICENSE", "GARBAGE"]

def just( image ) :
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    lab[:,:,0] = clahe.apply(lab[:,:,0])
    enhanced_img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    
    gray = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 700, 350, apertureSize = 5, L2gradient = True)
    lines = cv2.HoughLinesP(canny, 1, np.pi / 180, 50, minLineLength = 3, maxLineGap = 150)
    
    angle = 0
    maxdim = 0
    angles = []
    if not (lines is None):
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle_rad = math.atan2(y2 - y1, x2 - x1)
            angle_deg = math.degrees(angle_rad)
            angles.append(angle_deg)

    # 각도들을 히스토그램으로 표현하여 가장 많이 등장하는 각도를 찾습니다.
    histogram = cv2.calcHist([np.array(angles).astype(np.float32)], [0], None, [180], [-90, 90])
    max_angle_index = np.argmax(histogram)
    most_frequent_angle = max_angle_index - 90  # 각도 범위는 -90도부터 90도까지입니다.

    #print("가장 빈번하게 등장하는 각도:", most_frequent_angle)
    
    roih, roiw, roic = image.shape
    matrix = cv2.getRotationMatrix2D((roiw/2, roih/2), most_frequent_angle, 1)
    roi = cv2.warpAffine(image, matrix, (roiw, roih))
    #roi_org = cv2.warpAffine(image_org, matrix, (roiw, roih))
    return  roi

class MyWindow(QMainWindow, Ui_MainWindow) :
    def __init__(self) :
        super().__init__()
        self.IMG_PATH = None
        self.RK3588_RKNN_MODEL = None
        self.task = TASKS[0]
        self.DATASET = COCO
        self.input_size = 640
        self.dir_path = str(Path.cwd())
        self.yolo = None
        self.setupUi(self)
        self.lineEdit.setValidator(QIntValidator(320, 640, self))

    def model_select(self):
        dialog = QFileDialog(self)
        dialog.setWindowTitle("모델 열기")
        dialog.setNameFilter("Models (*.rknn)")
        dialog.setFileMode(QFileDialog.ExistingFile)
        dialog.setOption(QFileDialog.DontUseNativeDialog, True)  # Qt 다이얼로그 사용
        dialog.resize(400, 300)  # 다이얼로그 크기 축소
        # 시작 디렉토리 지정
        dialog.setDirectory(self.dir_path)

        # 커서 위치 (메뉴 클릭 위치)
        cursor_pos = QCursor.pos()

        # 다이얼로그 띄운 뒤 위치 이동
        def position_and_exec():
            dialog.move(cursor_pos)
            dialog.exec()

        QTimer.singleShot(0, position_and_exec)

        # 선택 파일 처리
        def on_file_selected():
            selected_files = dialog.selectedFiles()
            if selected_files:
                file_path = selected_files[0]
                #self.load_image_from_path(file_path)
                print( file_path )
                self.RK3588_RKNN_MODEL = file_path
                file_path = Path( file_path )
                self.pushButton_2.setText( file_path.name )
                name = os.path.splitext(file_path.name)[0]   # LicensePlate-RK3588_320_i8
                parts = name.split("_")
                print(parts)
                if "320" in parts:
                    print("해상도 320")
                    self.lineEdit.setText(str(320))
                elif "640" in parts:
                    print("해상도 640")
                    self.lineEdit.setText(str(640))
                
        dialog.fileSelected.connect(on_file_selected)

    def image_select(self):
        dialog = QFileDialog(self)
        dialog.setWindowTitle("이미지 열기")
        dialog.setNameFilter("Images (*.png *.jpg *.jpeg *.bmp *.gif)")
        dialog.setFileMode(QFileDialog.ExistingFile)
        dialog.setOption(QFileDialog.DontUseNativeDialog, True)  # Qt 다이얼로그 사용
        dialog.resize(400, 300)  # 다이얼로그 크기 축소
        # 시작 디렉토리 지정
        dialog.setDirectory(self.dir_path)

        # 커서 위치 (메뉴 클릭 위치)
        cursor_pos = QCursor.pos()

        # 다이얼로그 띄운 뒤 위치 이동
        def position_and_exec():
            dialog.move(cursor_pos)
            dialog.exec()

        QTimer.singleShot(0, position_and_exec)

        # 선택 파일 처리
        def on_file_selected():
            selected_files = dialog.selectedFiles()
            if selected_files:
                file_path = selected_files[0]
                #self.load_image_from_path(file_path)
                print( file_path )
                self.IMG_PATH = file_path
                file_path = Path(file_path)
                self.pushButton_3.setText( file_path.name )
                self.load_image_from_path(self.IMG_PATH)

        dialog.fileSelected.connect(on_file_selected)

    def load_image_from_path(self, file_path):
        image = cv2.imread(file_path)
        self.image, ratio = letterbox(image, 640, 480, 0)
        image_rgb = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        h, w, ch = image_rgb.shape
        bytes_per_line = ch * w
        qimage = QImage(image_rgb.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        self.label_4.setPixmap(pixmap)

    def task_select(self, index):
        #print(self.comboBox.itemText(index), index)
        self.task = TASKS[index]
        print( self.task)

    def dataset_select(self, index):
        #print(self.comboBox_3.itemText(index), index)
        self.DATASET = index
        print( self.DATASET )

    def cvimg_to_qpixmap(self, cv_img):
        image_rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        height, width, channel = image_rgb.shape
        bytes_per_line = channel * width
        qimg = QImage(image_rgb.data, width, height, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimg)
    
    def inference(self):
        print("infer click!!")
        if self.RK3588_RKNN_MODEL is None:
            print("RKNN Model not select!!")

            msg = QMessageBox(self)
            msg.setWindowTitle(" ")
            msg.setText("<b>경고</b>")
            msg.setInformativeText("모델이 선택되지 않았습니다.")
            msg.adjustSize()
            parent_center = self.geometry().center()
            msg_rect = msg.frameGeometry()
            msg_rect.moveCenter(parent_center)

            msg.move(msg_rect.topLeft())
            msg.setIcon(QMessageBox.Warning)
            msg.setStandardButtons(QMessageBox.Ok)
            msg.exec()
        elif self.IMG_PATH is None:
            print("Image not select!!")

            msg = QMessageBox(self)
            msg.setWindowTitle(" ")
            msg.setText("<b>경고</b>")
            msg.setInformativeText("이미지가 선택되지 않았습니다.")
            msg.adjustSize()
            parent_center = self.geometry().center()
            msg_rect = msg.frameGeometry()
            msg_rect.moveCenter(parent_center)

            msg.move(msg_rect.topLeft())
            msg.setIcon(QMessageBox.Warning)
            msg.setStandardButtons(QMessageBox.Ok)
            msg.exec()
        else:
            self.input_size = int(self.lineEdit.text())
            DS_NMS_THRESH = float(self.DS_NMS_VAL.text())
            DS_OBJ_THRESH = float(self.DS_OBJ_VAL.text())
            PO_NMS_THRESH = float(self.PO_NMS_VAL.text())
            PO_OBJ_THRESH = float(self.PO_OBJ_VAL.text())
            MAX_DETECT = int(self.MAX_DETECT_VAL.text())

            self.load_image_from_path(self.IMG_PATH)
            if self.yolo is not None:
                self.yolo.release()
            if self.task == "detect" :
                self.yolo = Yolo( "detect", 
                                self.RK3588_RKNN_MODEL, 
                                input_size = self.input_size, 
                                DATASET = self.DATASET,
                                DS_NMS_THRESH = DS_NMS_THRESH,
                                DS_OBJ_THRESH = DS_OBJ_THRESH,
                                PO_NMS_THRESH = PO_NMS_THRESH,
                                PO_OBJ_THRESH = PO_OBJ_THRESH,
                                MAX_DETECT = MAX_DETECT
                                )
                #img_org = cv2.imread(IMG_PATH)
                boxes, classes, scores = self.yolo.detect(self.image)
                if boxes is not None:
                    self.yolo.detect.draw(self.image, boxes, scores, classes, "all", label=True)
                    if self.DATASET == LICENSE:
                        print("LICENSE")
                        alpr = TFliteDemo('alpr_kor.tflite')
                        img_pil = Image.fromarray(self.image)
                        font = ImageFont.truetype("./fonts/NanumGothicBold.ttf",20)
                        draw = ImageDraw.Draw(img_pil)

                        count = 0
                        for box, score, cl in zip(boxes, scores, classes):
                            x1, y1, x2, y2 = box
                            detect_imge = self.image[y1:y2,x1:x2].copy()
                            just_image = just( detect_imge )
                            new_label, conf = alpr.run(just_image)
                            if conf > 0.6:
                                print( new_label )
                                draw.text((5, 20 + 20*count ),f'#{count} 번호판(License) : {new_label}', font=font, fill=(0, 0, 255))
                                count += 1
                        self.image = np.array(img_pil)
                    pixmap = self.cvimg_to_qpixmap(self.image)
                    self.label_4.setPixmap(pixmap)
                    self.lineEdit_7.setText(f'{self.yolo.detect.infertime:.4f}ms')
            elif self.task == "pose" :
                self.yolo = Yolo( "pose", 
                                self.RK3588_RKNN_MODEL, 
                                input_size = self.input_size, 
                                DATASET = self.DATASET,
                                DS_NMS_THRESH = DS_NMS_THRESH,
                                DS_OBJ_THRESH = DS_OBJ_THRESH,
                                PO_NMS_THRESH = PO_NMS_THRESH,
                                PO_OBJ_THRESH = PO_OBJ_THRESH,
                                MAX_DETECT = MAX_DETECT,
                                CALC_ANGLE=True
                                )
                predbox = self.yolo.pose(self.image)
                self.yolo.pose.draw(self.image, predbox, box_vis = False, angle=False)
                pixmap = self.cvimg_to_qpixmap(self.image)
                self.label_4.setPixmap(pixmap)
                self.lineEdit_7.setText(f'{self.yolo.pose.infertime:.4f}ms')
            elif self.task == "seg" :
                self.yolo = Yolo( "seg", 
                                self.RK3588_RKNN_MODEL, 
                                input_size = self.input_size, 
                                DATASET = self.DATASET,
                                DS_NMS_THRESH = DS_NMS_THRESH,
                                DS_OBJ_THRESH = DS_OBJ_THRESH,
                                PO_NMS_THRESH = PO_NMS_THRESH,
                                PO_OBJ_THRESH = PO_OBJ_THRESH,
                                MAX_DETECT = MAX_DETECT
                                )
                boxes, classes, scores, seg_img = self.yolo.seg(self.image)
                if boxes is not None:
                    img_p = self.yolo.seg.merge_seg(self.image, seg_img, classes)
                    self.yolo.seg.draw(img_p, boxes, scores, classes, "all", label=True)
                    pixmap = self.cvimg_to_qpixmap(img_p)
                    self.label_4.setPixmap(pixmap)
                    self.lineEdit_7.setText(f'{self.yolo.seg.infertime:.4f}ms')
                else:
                    print( "Box is None.")
            elif self.task == "obb" :
                self.yolo = Yolo( "obb", 
                                self.RK3588_RKNN_MODEL, 
                                input_size = self.input_size, 
                                DATASET = self.DATASET,
                                DS_NMS_THRESH = DS_NMS_THRESH,
                                DS_OBJ_THRESH = DS_OBJ_THRESH,
                                PO_NMS_THRESH = PO_NMS_THRESH,
                                PO_OBJ_THRESH = PO_OBJ_THRESH,
                                MAX_DETECT = MAX_DETECT
                                )
                predbox = self.yolo.obb(self.image)
                draw_img = self.yolo.obb.draw(predbox)
                pixmap = self.cvimg_to_qpixmap(draw_img)
                self.label_4.setPixmap(pixmap)
                self.lineEdit_7.setText(f'{self.yolo.obb.infertime:.4f}ms')


    def closeEvent(self, event):
        if self.yolo is not None:
            self.yolo.release()
        event.accept()  # 종료 계속 진행
        
if __name__ == "__main__":
    import sys
    if not QApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QApplication.instance()

    myWindow = MyWindow() 
    myWindow.show()
    sys.exit(app.exec())


seg
/home/darkice/ExtUSB/Work/rknn/rknn_examples/Yolo11_rknn/models/seg/yolov11_garbage/yolov11n_taco-RK3588_640_i8.rknn
['yolov11n', 'taco-RK3588', '640', 'i8']
해상도 640
4
/home/darkice/ExtUSB/Work/rknn/rknn_examples/Yolo11_rknn/images/garbage.jpg


/home/darkice/miniconda3/envs/pyside6rknn/lib/python3.12/site-packages/rknnlite/api/rknn_lite.py:41: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
W rknn-toolkit-lite2 version: 2.3.2
W Query dynamic range failed. Ret code: RKNN_ERR_MODEL_INVALID. (If it is a static shape RKNN model, please ignore the above warning message.)


infer click!!
YOLO : --> Load YOLO model
done
YOLO : --> Init runtime environment YOLO
I RKNN: [11:47:55.168] RKNN Runtime Information, librknnrt version: 2.3.2 (429f97ae6b@2025-04-09T09:09:27)
I RKNN: [11:47:55.168] RKNN Driver Information, version: 0.9.8
I RKNN: [11:47:55.169] RKNN Model Information, version: 6, toolkit version: 2.3.2(compiler version: 2.3.2 (e045de294f@2025-04-07T19:48:25)), target: RKNPU v2, target platform: rk3588, framework name: ONNX, framework layout: NCHW, model inference type: static_shape
W RKNN: [11:47:55.179] query RKNN_QUERY_INPUT_DYNAMIC_RANGE error, rknn model is static shape type, please export rknn with dynamic_shapes
done
YOLO : inference time : 37.92548179626465ms
Box is None.
YOLO : RKNN Seg Release!!


SystemExit: 0

/home/darkice/miniconda3/envs/pyside6rknn/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
